In [1]:
import numpy as np
from sklearn.mixture import GaussianMixture
from math import sqrt

# Deterministic Case 
In this case, the input is deterministic, and regardless of the shape of the tensor the result of the calculation is the same. With an input vector of y_pred and y_gt, it would return

In [2]:
#generate fake data
y_gt = np.random.randn(100,4,2)
y_pred = np.random.randn(100,4,2)
#calculate it 
np.sum( np.square(y_gt-y_pred)) # if reduction is sum
np.mean(np.square(y_gt-y_pred)) # if reduction is mean

1.9888486133306684

# Bivariate Case
If the input is bivariate, and the input prediction contains the values in the form 
$$  [ \mu_x, \mu_y, \rho, var_x, var_y ] $$
Then one would calculate the distance using the mahalobis distance in the following method:
1. First reshape the inputs to have the same number of rows (np.reshape was verified to be consistent)

In [38]:
ypb = np.random.rand(100,3,7,5) #generate y input
y_gt = np.random.rand(100,3,7,2) #generate ground truth
y_means = ypb.reshape(-1,5) #Reshape it into 2d
y_dat = y_gt.reshape(-1,2) #reshape it into 2d

2. Form the vector for the difference between the mean for the mahalanobis distance

In [39]:
means = np.stack( (y_dat[:,0] - y_means[:,0], y_dat[:,1] - y_means[:,1]) ,axis = 1)


In [40]:
means

array([[-0.44987614, -0.47171335],
       [-0.11302183, -0.05141372],
       [-0.63571928,  0.44535003],
       ...,
       [-0.05787041,  0.48233983],
       [-0.20424652,  0.57753605],
       [-0.23454623, -0.4488786 ]])

3. For each mean difference, calculate the Mahalanobis distance

In [41]:
total = 0
cv = np.zeros((2,2))
cov = np.zeros((2,2))  #Einsum 
for i in range(len(means)):
    v = means[i]
    cov[0,0] = y_means[i,3]
    cov[1,1] = y_means[i,4]
    cov[0,1] = cov[1,0] = np.sqrt(y_means[i,3]*y_means[i,4])*y_means[i,2]
    cv = np.concatenate((cv,cov))
    total+= sqrt((v.T @ np.linalg.inv(cov) @ v))
total

3662.4663635136026

### Bivariate Vectorized

In [42]:
cv = np.zeros((2,2))
cov = np.zeros((2,2))  #Einsum 
for i in range(len(means)):
    cov[0,0] = y_means[i,3]
    cov[1,1] = y_means[i,4]
    cov[0,1] = cov[1,0] = np.sqrt(y_means[i,3]*y_means[i,4])*y_means[i,2]
    cov = np.linalg.inv(cov)
    cv = np.concatenate((cv,cov))
cv = cv[2:] # all covariances
lv = np.repeat(means, repeats=2, axis = 0) #contains means duplicated
t = lv*cv
t = t.sum(axis=1)
t = means.flatten()* t
t = t[1::2] + t[::2] # take evens only, odds only, and sum corresponding ones
t = np.sqrt(t)
print(t.sum())

3662.4663635136094


# Multivariate case
This assumes a 5d tensor where the last two dimensions are in the form (samples, coordinates). For example, an input could be (200,100,5,100,2) where it is the batch, pedestrian, future step, samples, and coordinates of the samples. It will iterate through every group of samples, fit a gmm to it, and calculate a multivariate mahalanobis distance to the overall cluster center

In [6]:
total=0
samples = np.random.rand(200, 100,5,100, 2)
for i in samples:
    for pedestrian in i:
        for steps in pedestrian:
           #do the method of finding the best bic
            gmm = get_best_gmm(steps)
            center = np.sum(np.multiply(gmm.means_, gmm.weights_[:,np.newaxis]), axis = 0)
            dist = mahalanobis_d(center, true_value, len(gmm.weights_),gmm.covariances_, gmm.means_,gmm.weights_) #assume it will be the true value, add parameters
            total = total+dist
            #find the mahalanobis distance
            #For now compare the true mean to center of all the clusters
            
#total is reduction sum

[[0.02054451 0.00406262]
 [0.00406262 0.07809063]]
tied


NameError: name 'true_value' is not defined

The helper method to find the mahalanobis distance is below

In [3]:
def mahalanobis_d(x,y,n_clusters, ccov, cmeans,cluster_p): #ccov 
    v = np.array(x-y)
    Gnum = 0
    Gden = 0
    for i in range(0,n_clusters):
        ck = np.linalg.inv(ccov[i])
        u = np.array(cmeans[i] - y)
        val = ck*cluster_p[i]
        b2 =  1/(v.T @ ck @v)
        a = b2 * v.T @ ck @ u
        Z = u.T @ ck @ u - b2 * (v.T @ ck @ u)**2
        pxk = sqrt(np.pi*b2/2)*exp(-Z/2) * (erf((1-a)/sqrt(2*b2)) - erf(-a/sqrt(2*b2)))
        Gnum+= val*pxk 
        Gden += cluster_p[i] * pxk

    G = Gnum/Gden
    mdist = sqrt(v.T @ G @ v)
    print( "Mahalanobis distance between " + str(x) + " and "+str(y) + " is "+ str(mdist) )
    return mdist 

The method to find the best gmm uses BIC, and is below

In [4]:
def get_best_gmm(X):
    lowest_bic = np.infty
    bic = []
    n_components_range = range(1, 7)  ## stop based on fit/small BIC change/ earlystopping
    cv_types = ['spherical', 'tied', 'diag', 'full']
    best_gmm = GaussianMixture()
    for cv_type in cv_types:
        for n_components in n_components_range:
            # Fit a Gaussian mixture with EM
            gmm = GaussianMixture(n_components=n_components,
                                          covariance_type=cv_type)
            gmm.fit(X)
            bic.append(gmm.bic(X))
            if bic[-1] < lowest_bic :
                lowest_bic = bic[-1]
                best_gmm = gmm

    bic = np.array(bic)
    return best_gmm

### Early stopping GMM search

In [ ]:
#Early stopping version
def get_best_gmm2(X):
    lowest_bic = np.nfty
    bic = []
    n_components_range = range(1, 7)  ## stop based on fit/small BIC change/ earlystopping
    cv_types = ['full'] #changed to only looking for full covariance 
    best_gmm = GaussianMixture()
    for cv_type in cv_types:
        p=10    #Decide a value 
        n_comps = 1
        j = 0
        while j < p # if hasn't improved in p times, then stop. Do it for each cv type and take the minimum of all of them
            gmm = GaussianMixture(n_components=n_comps, covariance_type=cv_type)
            gmm.fit(X)
            bic.append(gmm.bic(X))
            if bic[-1] < lowest_bic:
                lowest_bic = bic[-1]
                best_gmm = gmm
                j = 0   #reset counter
            else: #increment counter
                j+=1

    bic = np.array(bic)
    return best_gmm